In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/megh/AI/Projects/Grapevine_Leaf_Classification'

In [81]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [82]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
from box import ConfigBox

In [83]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) :
                 
        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        self.config= ConfigBox(self.config[0]) # configbox is returning tuble so to avoid it we are again applying configbox to the dictionary at index 0
        self.params= ConfigBox(self.params[0])

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path), 
            params_image_size= self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes= self.params.CLASSES

        )

        return prepare_base_model_config


            

In [84]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [91]:
class PrepareBaseModel:
    def __init__(self, config : PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif(freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy'] 
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all= True,
            freeze_till=None,
            learning_rate= self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

In [92]:
from CNNClassifier import logger

In [93]:
try:

    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    logger.info(e)
    raise e


[2023-08-30 14:52:09,856: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
[2023-08-30 14:52:09,859: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-08-30 14:52:09,860: INFO: common: created directory at:artifacts]
[2023-08-30 14:52:09,861: INFO: common: created directory at:artifacts/prepare_base_model]
[2023-08-30 14:52:10,131: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      3

/home/megh/AI/Projects/Grapevine_Leaf_Classification/cnncls/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-08-30 14:42:11,564: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1zOY6cK2qJxdez2Vkrn_KEcvP6KjZYVbM/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}


{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1zOY6cK2qJxdez2Vkrn_KEcvP6KjZYVbM/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}
